## Introduction

In this part of the starter notebook, we will take you through the process of all three Subtasks.

In [ ]:
from IPython.display import clear_output
import time
import os

In [ ]:
! pip install -q gdown
clear_output()

In [ ]:
! mkdir experiments
! mkdir finetuned_models

## Imports

In [ ]:
dev_phase_id = "1tbAwUWN8X2JvXgdarjZ31f4XkqcpFVDk"
# subtask1_id = "1q_I6dw9ZbCg3MbQ1wnC-419s2ocCyqaa"
# subtask2_id = "1iHFDd_uihFi7vukWFq1hj32wfEH4dgBc"
# subtask3_id = "1JA7_BbJDYORbmH06gWzz4-UhXgRBe1eI"
translated_tasks_id = "1wHoKpZo8iMhHOm5TpvSS6Nr63Zk2w-P0"

! gdown --id $dev_phase_id
! gdown --id $translated_tasks_id
# ! gdown --id $subtask1_id
# ! gdown --id $subtask2_id
# ! gdown --id $subtask3_id

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1tbAwUWN8X2JvXgdarjZ31f4XkqcpFVDk
To: /content/dev_phase.zip
100% 18.9M/18.9M [00:00<00:00, 149MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1wHoKpZo8iMhHOm5TpvSS6Nr63Zk2w-P0
To: /content/translated_subtasks.zip
100% 1.93M/1.93M [00:00<00:00, 82.8MB/s]


In [ ]:
! unzip dev_phase.zip
! unzip translated_subtasks.zip
clear_output()

In [ ]:
# @title Module Imports

import numpy as np
import pandas as pd

from sklearn.metrics import recall_score, precision_score, f1_score
import numpy as np

import torch

from sklearn.metrics import f1_score

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from datasets import load_dataset, concatenate_datasets

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset

In [ ]:
import wandb

# Disable wandb logging for this script
wandb.init(mode="disabled")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


In [ ]:
from huggingface_hub import login
login()

# Functions & Classes

In [ ]:
# @title Dataset Class
# Fix the dataset class by inheriting from torch.utils.data.Dataset
class PolarizationDataset(torch.utils.data.Dataset):
  def __init__(self,texts,labels,tokenizer,max_length =128):
    self.texts=texts
    self.labels=labels
    self.tokenizer= tokenizer
    self.max_length = max_length # Store max_length

  def __len__(self):
    return len(self.texts)

  def __getitem__(self,idx):
    text=self.texts[idx]
    label=self.labels[idx]
    encoding=self.tokenizer(text,truncation=True,padding=False,max_length=self.max_length,return_tensors='pt')

    # Ensure consistent tensor conversion for all items
    item = {key: encoding[key].squeeze() for key in encoding.keys()}
    item['labels'] = torch.tensor(label, dtype=torch.long)
    return item

In [ ]:
# @title Data split function

import pandas as pd
import math
from sklearn.model_selection import StratifiedShuffleSplit


def load_and_split_bilingual_data(subtask, source_lang, target_lang, test_size=0.2, random_state=42, mode='train', verbose=True):
    """
    Load and split bilingual polarization data with stratified sampling.

    Args:
        subtask (str): Subtask name (e.g., 'subtask1', 'subtask2')
        source_lang (str): Source language code (e.g., 'swa', 'eng')
        target_lang (str): Target language code (e.g., 'eng', 'swa')
        test_size (float): Proportion of validation set (default: 0.2)
        random_state (int): Random seed for reproducibility (default: 42)
        verbose (bool): Print distribution statistics (default: True)

    Returns:
        tuple: (train_df, val_df) with columns:
            - source_text
            - polarization (source label)
            - target_text
            - target_polarization (target label)
    """

    # Load the DataFrames
    source = pd.read_csv(f'{subtask}/{mode}/{source_lang}.csv')
    target = pd.read_csv(f'{subtask}/{mode}/{target_lang}.csv')

    # --- 1. Shape Matching and Alignment ---
    source_len = source.shape[0]
    target_len = target.shape[0]

    if target_len < source_len:
        # Case A: Target is shorter than Source (Repeat/Tile the Target)
        repeat_factor = math.ceil(source_len / target_len)
        target_aligned = pd.concat([target] * repeat_factor, ignore_index=True).iloc[0:source_len]

    elif target_len > source_len:
        # Case B: Target is longer than Source (Truncate the Target)
        target_aligned = target.iloc[0:source_len]

    else:
        # Case C: Target and Source are already the same length
        target_aligned = target.copy()

    # --- 2. Data Combination ---
    data = pd.DataFrame({
        'source_text': source['text'],
        'polarization': source['polarization'],
        'target_text': target_aligned['text'],
        'target_polarization': target_aligned['polarization']
    })

    # --- 3. Stratified Splitting ---
    sss_source = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)

    # Split based on source polarization
    for train_index, val_index in sss_source.split(data, data['polarization']):
        train = data.iloc[train_index].reset_index(drop=True)
        val = data.iloc[val_index].reset_index(drop=True)

    # Stratified split for target language in validation set
    full_target_pool = data[['target_text', 'target_polarization']].copy()
    val_size = val.shape[0]
    test_ratio = val_size / data.shape[0]

    sss_target = StratifiedShuffleSplit(n_splits=1, test_size=test_ratio, random_state=random_state)
    for _, new_val_target_index in sss_target.split(full_target_pool, full_target_pool['target_polarization']):
        new_val_target = full_target_pool.iloc[new_val_target_index].reset_index(drop=True)

    # Verify and update validation target data
    if new_val_target.shape[0] == val.shape[0]:
        val['target_text'] = new_val_target['target_text'].values
        val['target_polarization'] = new_val_target['target_polarization'].values
    else:
        raise ValueError(f"Stratified target split size ({new_val_target.shape[0]}) "
                        f"does not match validation set size ({val.shape[0]}).")

    # --- 4. Print Results (if verbose) ---
    if verbose:
        print("="*60)
        print(f"Dataset: {subtask} | {source_lang} → {target_lang}")
        print("="*60)
        print(f"Total samples: {data.shape[0]}")
        print(f"Train set size: {train.shape[0]} ({train.shape[0]/data.shape[0]*100:.1f}%)")
        print(f"Validation set size: {val.shape[0]} ({val.shape[0]/data.shape[0]*100:.1f}%)")

        print("\n--- Polarization Distribution ---")
        print("\nTrain Set (Source):")
        print(train['polarization'].value_counts().sort_index())
        print(f"  Ratio: {train['polarization'].value_counts(normalize=True).sort_index().to_dict()}")

        print("\nValidation Set (Source):")
        print(val['polarization'].value_counts().sort_index())
        print(f"  Ratio: {val['polarization'].value_counts(normalize=True).sort_index().to_dict()}")

        print("\nValidation Set (Target):")
        print(val['target_polarization'].value_counts().sort_index())
        print(f"  Ratio: {val['target_polarization'].value_counts(normalize=True).sort_index().to_dict()}")
        print("="*60)

    return train, val

In [ ]:
# @title Metrics

# Define metrics function
def subtask1_codabench_compute_metrics(p):
  preds = np.argmax(p.predictions, axis=1)
  return {'f1_macro': f1_score(p.label_ids, preds, average='macro')}

def subtask2_codabench_compute_metrics_multilabel(p):
  # Sigmoid the predictions to get probabilities
  probs = torch.sigmoid(torch.from_numpy(p.predictions))
  # Convert probabilities to predicted labels (0 or 1)
  preds = (probs > 0.5).int().numpy()
  # Compute macro F1 score
  return {'f1_macro': f1_score(p.label_ids, preds, average='macro')}



# def compute_metrics(p):
#     print("=" * 50)
#     print(f"Type of p.predictions: {type(p.predictions)}")
#     print(f"Type of p.label_ids: {type(p.label_ids)}")

#     if isinstance(p.predictions, tuple):
#         print(f"Predictions is tuple with {len(p.predictions)} elements")
#         for i, item in enumerate(p.predictions):
#             print(f"  Element {i}: {type(item)}, shape: {getattr(item, 'shape', 'no shape')}")
#         logits = p.predictions[0]
#     elif isinstance(p.predictions, np.ndarray):
#         print(f"Predictions shape: {p.predictions.shape}")
#         logits = p.predictions
#     else:
#         print(f"Predictions: {p.predictions}")
#         logits = np.array(p.predictions)

#     print(f"Label_ids shape: {p.label_ids.shape}")
#     print(f"Logits shape after extraction: {logits.shape}")

#     preds = np.argmax(logits, axis=1)
#     print(f"Preds shape after argmax: {preds.shape}")
#     print("=" * 50)

#     return {'f1_macro': f1_score(p.label_ids, preds, average='macro')}

# def compute_metrics(p):
#     # Handle predictions
#     logits = p.predictions
#     preds = np.argmax(logits, axis=1)

#     # Handle labels - convert tuple to array if needed
#     if isinstance(p.label_ids, tuple):
#         labels = np.array(p.label_ids)
#     else:
#         labels = p.label_ids

#     # Flatten if needed
#     if labels.ndim > 1:
#         labels = labels.flatten()

#     return {'f1_macro': f1_score(labels, preds, average='macro')}

def compute_metrics(p):
    """Simple, robust compute_metrics function"""

    # Predictions should be (n_samples, n_classes)
    logits = p.predictions
    preds = np.argmax(logits, axis=1)

    # Labels should be (n_samples,)
    labels = p.label_ids

    # Ensure labels are 1D
    if isinstance(labels, (list, tuple)):
        labels = np.array(labels)

    if labels.ndim > 1:
        labels = labels.flatten()

    # Calculate F1
    return {
        'f1_macro': f1_score(labels, preds, average='macro'),
        'f1_weighted': f1_score(labels, preds, average='weighted')
    }

In [ ]:
# @title Experiment Class

from typing import List

import yaml
import numpy as np
import torch


class Experiment:

  def __init__(self, name, dir, description, baseline=None):
    self.name = name
    self.dir = dir
    self.description = description
    self.parameters = dict()
    self.baseline = baseline

  def update_param(self, parameter: 'Parameter'):
    var_name = parameter.get_var_name()
    parameter_class = parameter.get_parameter_class()
    value = parameter.get_value()

    assert isinstance(
        value,
        (
            int, float, str, dict, list,
            np.ndarray, torch.tensor
        )
    )

    if (parameter_class is None) or (parameter_class.lower() == 'global'):
      self.parameters[var_name] = value
      return

    if parameter_class not in self.parameters:
      self.parameters[parameter_class] = dict()
    self.parameters[parameter_class][var_name] = value

  def save(self):
    experiment_dict = {
        'name': self.name,
        'baseline': self.baseline,
        'description': self.description,
        'parameters': self.parameters,
    }

    with open(self.dir, "w") as f:
      yaml.dump(experiment_dict, f, default_flow_style=False)

    print(f"Model saved to {self.dir}")

  def add_params(self, parameters: List['Parameter']):

    for parameter in parameters:
      self.update_param(parameter)


class Parameter:

  def __init__(self, value, var_name, parameter_class):
    self.__var_name = var_name
    self.__value = value
    self.__parameter_class = parameter_class

  def get_var_name(self):
    return self.__var_name

  def get_parameter_class(self):
    return self.__parameter_class

  def get_value(self):
    return self.__value

  def set_value(self, value):
    self.__value = value

  def set_var_name(self, var_name):
    self.__var_name = var_name

  def set_parameter_class(self, parameter_class):
    self.__parameter_class = parameter_class

In [ ]:
# @title PolarPairs Approach

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from transformers import AutoModel, AutoTokenizer, Trainer
import numpy as np

class PolarPairsDataset(Dataset):
    """Combined dataset with stratified target sampling"""
    def __init__(self, source_texts, target_texts, source_labels, target_labels,
                 source_tokenizer, target_tokenizer, max_length=128, subset_size=20):
        self.source_labels = source_labels
        self.target_labels = np.array(target_labels)
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.source_tokenizer = AutoTokenizer.from_pretrained(source_tokenizer)
        self.target_tokenizer = AutoTokenizer.from_pretrained(target_tokenizer)
        self.max_length = max_length
        self.subset_size = subset_size

        # Pre-compute indices by class for stratified sampling
        self.pos_indices = np.where(self.target_labels == 1)[0]
        self.neg_indices = np.where(self.target_labels == 0)[0]

        self.mode = 'train'

        print(f"Target distribution - Positive: {len(self.pos_indices)}, "
              f"Negative: {len(self.neg_indices)}")

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        source = str(self.source_texts[idx])
        source_label = int(self.source_labels[idx])

        # Stratified sampling: ensure balanced positive/negative targets
        n_pos = self.subset_size // 2
        n_neg = self.subset_size - n_pos

        # Sample with replacement if needed
        pos_sample = np.random.choice(self.pos_indices, size=n_pos,
                                     replace=len(self.pos_indices) < n_pos)
        neg_sample = np.random.choice(self.neg_indices, size=n_neg,
                                     replace=len(self.neg_indices) < n_neg)

        target_indices = np.concatenate([pos_sample, neg_sample])
        np.random.shuffle(target_indices)

        target_texts_batch = [str(self.target_texts[i]) for i in target_indices]
        target_labels_batch = [int(self.target_labels[i]) for i in target_indices]

        # Encode
        source_encoding = self.source_tokenizer(
            source, max_length=self.max_length, padding='max_length',
            truncation=True, return_tensors='pt'
        )

        target_encoding = self.target_tokenizer(
            target_texts_batch, max_length=self.max_length, padding='max_length',
            truncation=True, return_tensors='pt'
        )

        if self.mode == 'train':
          return {
              'x_input_ids': source_encoding['input_ids'].squeeze(0),
              'x_attention_mask': source_encoding['attention_mask'].squeeze(0),
              'x_hat_input_ids': target_encoding['input_ids'],
              'x_hat_attention_mask': target_encoding['attention_mask'],
              'polar_labels': torch.tensor(source_label, dtype=torch.long),
              'hat_labels': torch.tensor(target_labels_batch, dtype=torch.long)
          }
        else:
          return {
              'x_input_ids': source_encoding['input_ids'].squeeze(0),
              'x_attention_mask': source_encoding['attention_mask'].squeeze(0),
              'polar_labels': torch.tensor(source_label, dtype=torch.long),
          }

class MultiLingualPolarPairsAlignment(nn.Module):

  def __init__(self, encoder_model_name, pretrained_encoder_name, num_labels, alignment_normalization=True, alignment_residual=True):
    super(MultiLingualPolarPairsAlignment, self).__init__()

    self.encoder = AutoModel.from_pretrained(encoder_model_name)
    self.encoder.train()
    self.pretrained_encoder = AutoModel.from_pretrained(pretrained_encoder_name)
    self.pretrained_encoder.eval()

    encoder_config = self.encoder.config
    encoder_embedding_size = encoder_config.hidden_size

    pretrained_encoder_config = self.pretrained_encoder.config
    pretrained_encoder_embedding_size = pretrained_encoder_config.hidden_size

    self.alignment_head = nn.Linear(encoder_embedding_size, pretrained_encoder_embedding_size)
    self.classification_head = nn.Linear(pretrained_encoder_embedding_size, num_labels)
    self.alignment_layer_norm = nn.LayerNorm(pretrained_encoder_embedding_size)
    self.alignment_normalization = alignment_normalization
    self.alignment_residual = True

  def get_cls_embeddings(self, x_input_ids, x_attention_mask, polar_labels):
    x1_hidden = self.encoder(
        input_ids=x_input_ids,
        attention_mask=x_attention_mask
    ).last_hidden_state
    return x1_hidden[:, 0, :]

  def cls_resnet(self, x):
    return self.alignment_head(x) + x

  def cls_normalization(self, x):
    return self.alignment_layer_norm(x)

  def get_aligned_embedding(self, x_input_ids, x_attention_mask, polar_labels):
    x_cls = self.get_cls_embeddings(x_input_ids, x_attention_mask, polar_labels)

    if self.alignment_residual:
      x_cls = self.cls_resnet(x_cls)
    else:
      x_cls = self.alignment_head(x_cls)

    if self.alignment_normalization:
      x_cls = self.cls_normalization(x_cls)

    return x_cls

  def forward(self, x_input_ids, x_attention_mask, x_hat_input_ids, x_hat_attention_mask, polar_labels, hat_labels):
    # Get Aligned Embedding for source language
    x1_aligned = self.get_aligned_embedding(x_input_ids, x_attention_mask, polar_labels)

    # Flatten targets before encoding
    batch_size, subset_size, seq_len = x_hat_input_ids.shape
    x_hat_input_ids_flat = x_hat_input_ids.view(batch_size * subset_size, seq_len)
    x_hat_attention_mask_flat = x_hat_attention_mask.view(batch_size * subset_size, seq_len)
    x2_hidden_flat = self.pretrained_encoder(
        input_ids=x_hat_input_ids_flat,
        attention_mask=x_hat_attention_mask_flat
    ).last_hidden_state  # ✓ Shape: (batch_size * subset_size, seq_len, hidden_size)

    # Reshape back correctly
    hidden_size = x2_hidden_flat.size(-1)
    x2_hidden = x2_hidden_flat.view(batch_size, subset_size, seq_len, hidden_size)
    # Extract CLS token
    x2_cls = x2_hidden[:, :, 0, :]  # (batch_size, subset_size, hidden_size)

    # # Apply alignment to all target embeddings
    # batch_size, subset_size, hidden_size = x2_cls.shape
    # x2_cls_flat = x2_cls.view(batch_size * subset_size, hidden_size)
    # x2_aligned_flat = F.leaky_relu(self.alignment_head(x2_cls_flat)) + x2_cls_flat
    # if self.alignment_normalization:
    #   x2_aligned_flat = self.alignment_layer_norm(x2_cls_flat + x2_aligned_flat)
    # x2_aligned = x2_aligned_flat.view(batch_size, subset_size, hidden_size)

    # Use target embeddings directly (no transformation)
    x2_aligned = x2_cls  # (batch_size, subset_size, hidden_size)

    # Classification logits (only from source)
    logits = self.classification_head(x1_aligned)

    return logits, x1_aligned, x2_aligned

  def compute_embeddings(self, x_input_ids, x_attention_mask, polar_labels):
    x_embd = self.get_aligned_embedding(x_input_ids, x_attention_mask, polar_labels)
    return {
        'embeddings': x_embd,
        'labels': polar_labels
    }

  def compute_logits(self, x_input_ids, x_attention_mask, polar_labels):
    embeddings = self.compute_embeddings(x_input_ids, x_attention_mask, polar_labels)
    logits = self.compute_embeddings(embeddings['embeddings'])
    return {
        'logits': logits,
        'labels': polar_labels
    }

def polar_pairs_contrastive_loss(
    logits: torch.Tensor,
    x1_aligned: torch.Tensor,
    x2_aligned: torch.Tensor,
    polar_labels: torch.Tensor,
    hat_labels: torch.Tensor,
    lambda_align: float = 0.5,
    temperature: float = 0.07
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Improved loss with:
    1. Classification loss
    2. Contrastive alignment loss (pull same class together, push different apart)
    3. Class balance handling
    """

    # 1. Classification loss with class weights
    num_classes = logits.size(1)  # Get number of classes from logits
    class_counts = torch.bincount(polar_labels, minlength=num_classes)
    class_weights = 1.0 / (class_counts.float() + 1e-6)
    class_weights = class_weights / class_weights.sum()

    # Move weights to same device as logits
    class_weights = class_weights.to(logits.device)

    classification_loss = F.cross_entropy(logits, polar_labels, weight=class_weights)

    # 2. Contrastive alignment loss
    batch_size, subset_size, hidden_size = x2_aligned.shape

    # Expand source embeddings to match all targets
    x1_expanded = x1_aligned.unsqueeze(1).expand(-1, subset_size, -1)
    x1_flat = x1_expanded.reshape(batch_size * subset_size, hidden_size)
    x2_flat = x2_aligned.reshape(batch_size * subset_size, hidden_size)

    # Expand source labels to match
    source_labels_expanded = polar_labels.unsqueeze(1).expand(-1, subset_size)
    source_labels_flat = source_labels_expanded.reshape(-1)
    target_labels_flat = hat_labels.reshape(-1)

    # Compute similarity matrix
    x1_norm = F.normalize(x1_flat, p=2, dim=1)
    x2_norm = F.normalize(x2_flat, p=2, dim=1)
    similarity = torch.mm(x1_norm, x2_norm.t()) / temperature

    # Create masks for positive and negative pairs
    labels_equal = (source_labels_flat.unsqueeze(1) == target_labels_flat.unsqueeze(0))

    # InfoNCE-style contrastive loss
    exp_sim = torch.exp(similarity)

    # Mask out self-similarities (diagonal)
    mask_self = torch.eye(similarity.size(0), device=similarity.device).bool()
    exp_sim = exp_sim.masked_fill(mask_self, 0)

    # Positive pairs: same label
    pos_sim = (exp_sim * labels_equal.float()).sum(dim=1)

    # All pairs (excluding self)
    all_sim = exp_sim.sum(dim=1)

    # Contrastive loss: -log(pos / (pos + neg))
    alignment_loss = -torch.log((pos_sim + 1e-8) / (all_sim + 1e-8))
    alignment_loss = alignment_loss.mean()

    # 3. Total loss
    total_loss = (1.0 - lambda_align) * classification_loss + lambda_align * alignment_loss

    return total_loss, classification_loss, alignment_loss


class PolarPairsTrainer(Trainer):
    def __init__(self, lambda_align=0.5, temperature=0.07, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.lambda_align = lambda_align
        self.temperature = temperature
        self.label_names = ['polar_labels', 'hat_labels']

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        logits, x1_aligned, x2_aligned = model(
            x_input_ids=inputs['x_input_ids'],
            x_attention_mask=inputs['x_attention_mask'],
            x_hat_input_ids=inputs['x_hat_input_ids'],
            x_hat_attention_mask=inputs['x_hat_attention_mask'],
            polar_labels=inputs['polar_labels'],
            hat_labels=inputs['hat_labels']
        )

        total_loss, classification_loss, alignment_loss = polar_pairs_contrastive_loss(
            logits=logits,
            x1_aligned=x1_aligned,
            x2_aligned=x2_aligned,
            polar_labels=inputs['polar_labels'],
            hat_labels=inputs['hat_labels'],
            lambda_align=self.lambda_align,
            temperature=self.temperature
        )

        # Log component losses
        if self.state.global_step % 10 == 0:
            self.log({
                'classification_loss': classification_loss.item(),
                'alignment_loss': alignment_loss.item(),
                'lambda': self.lambda_align
            })

        if return_outputs:
            outputs = {'logits': logits}
            return total_loss, outputs
        else:
            return total_loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        with torch.no_grad():
            loss, outputs = self.compute_loss(model, inputs, return_outputs=True)
            logits = outputs['logits']

        labels = inputs['polar_labels'].detach().cpu()

        if prediction_loss_only:
            return (loss.detach(), None, None)

        return (loss.detach(), logits.detach().cpu(), labels)


# Enhanced collator
from dataclasses import dataclass
from typing import Any, Dict, List

@dataclass
class PolarPairsCollator:
    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        batch = {}

        batch['x_input_ids'] = torch.stack([f['x_input_ids'] for f in features])
        batch['x_attention_mask'] = torch.stack([f['x_attention_mask'] for f in features])
        batch['x_hat_input_ids'] = torch.stack([f['x_hat_input_ids'] for f in features])
        batch['x_hat_attention_mask'] = torch.stack([f['x_hat_attention_mask'] for f in features])

        polar_labels = []
        for f in features:
            label = f['polar_labels']
            if isinstance(label, torch.Tensor):
                polar_labels.append(label.item() if label.dim() == 0 else label[0].item())
            else:
                polar_labels.append(int(label))

        batch['polar_labels'] = torch.tensor(polar_labels, dtype=torch.long)
        batch['hat_labels'] = torch.stack([f['hat_labels'] for f in features])

        return batch

In [ ]:
# @title Knowledge Distillation Classes & Functions

def load_flores_multilingual(languages=None):
    """
    Load FLORES-200 for specified languages using the new format
    """
    if languages is None:
        # Default: African languages + English
        languages = [
            "yor_Latn",  # Yoruba
            "hau_Latn",  # Hausa
            "ibo_Latn",  # Igbo
            "amh_Ethi",  # Amharic
            "swh_Latn",  # Swahili
            "zul_Latn",  # Zulu
            "som_Latn",  # Somali
            "kin_Latn",  # Kinyarwanda
            "eng_Latn",  # English
        ]

    all_sentences = []

    # NEW METHOD: Load the entire dataset once, then filter by language
    try:
        print("Loading FLORES-200 dataset (this may take a moment)...")
        # Load without trust_remote_code since scripts are deprecated
        dataset = load_dataset("facebook/flores", "all", trust_remote_code=False)

        print(f"Dataset loaded. Available splits: {list(dataset.keys())}")

        # Process each split
        for split in ['dev', 'devtest']:
            if split in dataset:
                split_data = dataset[split]

                # Check available columns
                print(f"\n{split} columns: {split_data.column_names}")

                # Extract sentences for each language
                for lang in languages:
                    # The column name might be different - check if it exists
                    if lang in split_data.column_names:
                        sentences = split_data[lang]
                        all_sentences.extend(sentences)
                        print(f"Loaded {len(sentences)} sentences for {lang} ({split})")
                    else:
                        print(f"Column {lang} not found in {split}")

    except Exception as e:
        print(f"Error with 'all' method: {e}")
        print("\nTrying alternative loading method...")

        # ALTERNATIVE METHOD: Use the FLORES+ dataset which is newer
        try:
            for lang in languages:
                # Try loading individual language configs
                try:
                    data = load_dataset("openlanguagedata/flores_plus", lang, trust_remote_code=False)

                    for split in ['dev', 'devtest']:
                        if split in data:
                            sentences = data[split]['text']
                            all_sentences.extend(sentences)
                            print(f"Loaded {len(sentences)} sentences for {lang} ({split})")
                except Exception as lang_error:
                    print(f"Could not load {lang}: {lang_error}")

        except Exception as e2:
            print(f"Alternative method also failed: {e2}")

    print(f"\nTotal sentences: {len(all_sentences)}")

    # If still no data, use a fallback dataset
    if len(all_sentences) == 0:
        print("\n⚠️  FLORES loading failed. Using alternative dataset: NLLB...")
        all_sentences = load_nllb_fallback(languages)

    return all_sentences

# 3. Fallback: Load NLLB data if FLORES fails
def load_nllb_fallback(languages):
    """Fallback to NLLB dataset if FLORES doesn't work"""
    all_sentences = []

    # Map language codes
    lang_map = {
        "amh_Ethi": "amh_Ethi",
        "swa_Latn": "swh_Latn",  # Note: NLLB uses 'swh' not 'swa'
        "hau_Latn": "hau_Latn",
        "yor_Latn": "yor_Latn",
        "ibo_Latn": "ibo_Latn",
        "zul_Latn": "zul_Latn",
        "som_Latn": "som_Latn",
        "kin_Latn": "kin_Latn",
        "eng_Latn": "eng_Latn",
    }

    try:
        # Load NLLB which has similar multilingual data
        for flores_code, nllb_code in lang_map.items():
            try:
                # Load parallel data with English
                dataset = load_dataset("allenai/nllb", f"{nllb_code}-eng_Latn", split="train")

                # Take first 2000 samples
                for i, item in enumerate(dataset):
                    if i >= 2000:
                        break
                    all_sentences.append(item['translation'][nllb_code])

                print(f"Loaded 2000 sentences for {flores_code} from NLLB")
            except Exception as e:
                print(f"Could not load {flores_code} from NLLB: {e}")

    except Exception as e:
        print(f"NLLB fallback failed: {e}")

    return all_sentences

# Dataset to use Teacher and Student tokenizers
class DualTokenizerDataset(Dataset):
  def __init__(self, sentences, teacher_tokenizer, student_tokenizer, max_length=128):
    self.sentences = sentences
    self.teacher_tokenizer = teacher_tokenizer
    self.student_tokenizer = student_tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.sentences)

  def __getitem__(self, idx):
    sentence = self.sentences[idx]

    # Tokenize with BOTH tokenizers
    teacher_encoded = self.teacher_tokenizer(
      sentence,
      padding='max_length',
      truncation=True,
      max_length=self.max_length,
      return_tensors='pt'
    )

    student_encoded = self.student_tokenizer(
      sentence,
      padding='max_length',
      truncation=True,
      max_length=self.max_length,
      return_tensors='pt'
    )

    return {
      # Student inputs (for training)
      'input_ids': student_encoded['input_ids'].squeeze(0),
      'attention_mask': student_encoded['attention_mask'].squeeze(0),
      # Teacher inputs (for reference)
      'teacher_input_ids': teacher_encoded['input_ids'].squeeze(0),
      'teacher_attention_mask': teacher_encoded['attention_mask'].squeeze(0)
    }


def mean_pooling(model_output, attention_mask):
  token_embeddings = model_output[0]
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
  return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Updated Trainer
class EmbeddingDistillationTrainer(Trainer):
  def __init__(self, *args, teacher_model=None, **kwargs):
    super().__init__(*args, **kwargs)
    self.teacher = teacher_model
    self.mse_loss = nn.MSELoss()

    self.teacher.eval()
    for param in self.teacher.parameters():
        param.requires_grad = False

  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
    # Extract teacher inputs
    teacher_inputs = {
      'input_ids': inputs.pop('teacher_input_ids'),
      'attention_mask': inputs.pop('teacher_attention_mask')
    }

    # Student embeddings (uses student tokenizer inputs)
    student_outputs = model(**inputs)
    student_embeddings = mean_pooling(student_outputs, inputs['attention_mask'])
    student_embeddings = F.normalize(student_embeddings, p=2, dim=-1)

    # Teacher embeddings (uses teacher tokenizer inputs)
    with torch.no_grad():
      teacher_outputs = self.teacher(**teacher_inputs)
      teacher_embeddings = mean_pooling(teacher_outputs, teacher_inputs['attention_mask'])
      teacher_embeddings = F.normalize(teacher_embeddings, p=2, dim=-1)

    # MSE loss between normalized embeddings
    loss = self.mse_loss(student_embeddings, teacher_embeddings)

    return (loss, student_outputs) if return_outputs else loss

# Subtask 1 - Polarization detection

This is a binary classification to determine whether a post contains polarized content (Polarized or Not Polarized).

In [ ]:
# @title Experiment Definition & Parameter Setting

import os

languages = ['amh', 'eng', 'hau', 'swa'] # @param
languages_param = Parameter(languages, 'language', None)

model_name = "olateju/PolarPairs-v1.3.6.3" # @param {type: "string"}
model_param = Parameter(model_name, 'model_name', None)

tokenizer_choice = "olateju/PolarPairs-v1.3.6.3" # @param {type:"string"}
tokenizer_param = Parameter(tokenizer_choice, 'tokenizer', 'Preprocessing')

n_labels = 2 # @param
n_labels_param = Parameter(n_labels, 'n_labels', None)

max_length = 128 # @param
max_length_param = Parameter(max_length, 'max_length', 'Hyperparameter')

num_epochs = 3 # @param
epochs_param = Parameter(num_epochs, 'epochs', 'Hyperparameter')

lr = 1e-4 # @param
lr_param = Parameter(lr, 'learning_rate', 'Hyperparameter')

train_batch = 8 # @param
train_batch_param = Parameter(train_batch, 'train_batch_size', 'Hyperparameter')

eval_batch = 8 # @param
eval_batch_param = Parameter(eval_batch, 'eval_batch_size', 'Hyperparameter')

eval_strategy = "epoch" # @param {type: "string"}
eval_strategy_param = Parameter(eval_strategy, 'eval_strategy', 'Hyperparameter')

experiment_version = "v1.3.3.3" # @param {type: "string"}
experiment_dir = f"experiments/{experiment_version}.yaml"
if os.path.exists(experiment_dir):
  raise ValueError(f"Experiment {experiment_version} already exists")
experiment_description = "Matching Encoder Pairs: distilled-BERT from LaBSE on African languages as slave-model and english optimized distilled-BERT as anchor-model" # @param {type: "string"}
experiment_baseline = "v1.0.6, v1.1.6, v1.2.6.*" # @param {type: "string"}
experiment = Experiment(
    experiment_version,
    experiment_dir,
    experiment_description,
    experiment_baseline
)
experiment.add_params([
    languages_param, model_param, max_length_param,
    tokenizer_param, epochs_param, lr_param,
    train_batch_param, eval_batch_param,
    eval_strategy_param
])

save_models = False # @param {type: 'boolean'}

In [ ]:
# @title Training Setup
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs_param.get_value(),
    per_device_train_batch_size=train_batch_param.get_value(),  # mT5-small can handle 2-4
    per_device_eval_batch_size=eval_batch_param.get_value(),
    gradient_accumulation_steps=8,
    eval_strategy=eval_strategy_param.get_value(),
    save_strategy='no',
    logging_steps=50,
    fp16=True,
    dataloader_num_workers=0,
    load_best_model_at_end=False,
    eval_accumulation_steps=1,
    gradient_checkpointing=False,
    # metric_for_best_model="eval_loss"
)

In [ ]:
# source = pd.read_csv(f'subtask1/train/amh.csv')
# target = pd.read_csv(f'subtask1/train/eng.csv')

# train_stop_idx = int(0.8*source.shape[0])

# data = pd.DataFrame()
# data['source_text'] = source['text']
# data['polarization'] = source['polarization']
# data['target_text'] = target['text']
# data['target_polarization'] = target['polarization']

# train = data[0: train_stop_idx]
# val = data[train_stop_idx: ]

#### Test Model Per Language

In [ ]:
# @title Search for Base Model (English to be prioritized)

np.random.seed(42)
torch.manual_seed(42)

for language in languages_param.get_value():
  time.sleep(0.25)
  clear_output()

  # Load the tokenizer
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_param.get_value())

  train, val = load_and_split_bilingual_data(
      subtask = 'subtask1',
      source_lang = language,
      target_lang = 'eng'
  )

  # Create datasets
  train_dataset = PolarizationDataset(train['source_text'].tolist(), train['polarization'].tolist(), tokenizer)
  val_dataset = PolarizationDataset(val['source_text'].tolist(), val['polarization'].tolist(), tokenizer)

  # Load the model
  model = AutoModelForSequenceClassification.from_pretrained(
      tokenizer_param.get_value(),
      num_labels=n_labels_param.get_value(),
      ignore_mismatched_sizes=True
  )

  # Initialize the Trainer
  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset,            # evaluation dataset
      compute_metrics=subtask1_codabench_compute_metrics,     # the callback that computes metrics of interest
      data_collator=DataCollatorWithPadding(tokenizer), # Data collator for dynamic padding
      # callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
  )

  # Train the model
  trainer.train()

  eval_results = trainer.evaluate()
  print(f"Macro F1 score on {language} validation set: {eval_results['eval_f1_macro']}")

  if save_models:
    model.push_to_hub(f"olateju/PolarPairs-{experiment_version}_{language}")
    tokenizer.push_to_hub(f"olateju/PolarPairs-{experiment_version}_{language}")

  # ===== SAVE THE FINE-TUNED MODEL =====
  # save_path = f'finetuned_models/{language}_{tokenizer_param.get_value()}'
  # model.save_pretrained(save_path)
  # tokenizer.save_pretrained(save_path)
  # print(f"Saved {language} model to {save_path}")
  # ====================================

  eval_results_param = Parameter(eval_results, f"{language}_eval_results", "Performance")
  experiment.add_params([eval_results_param])

Dataset: subtask1 | swa → eng
Total samples: 6991
Train set size: 5592 (80.0%)
Validation set size: 1399 (20.0%)

--- Polarization Distribution ---

Train Set (Source):
polarization
0    2789
1    2803
Name: count, dtype: int64
  Ratio: {0: 0.49874821173104433, 1: 0.5012517882689557}

Validation Set (Source):
polarization
0    698
1    701
Name: count, dtype: int64
  Ratio: {0: 0.498927805575411, 1: 0.501072194424589}

Validation Set (Target):
target_polarization
0    929
1    470
Name: count, dtype: int64
  Ratio: {0: 0.6640457469621158, 1: 0.3359542530378842}


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


Epoch,Training Loss,Validation Loss,F1 Macro
1,0.585400,0.493035,0.751202
2,0.413900,0.477973,0.771214
3,0.326800,0.507060,0.778388


Macro F1 score on swa validation set: 0.7783876756792112


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...tkl4vyj/model.safetensors:   6%|5         | 31.8MB /  541MB            

README.md: 0.00B [00:00, ?B/s]

In [ ]:
# @title Codabench Submission

! mkdir -p results/subtask_1/

from torch.utils.data import DataLoader
from tqdm import tqdm

for language in languages_param.get_value():
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_param.get_value())
  model = AutoModelForSequenceClassification.from_pretrained(
      f'finetuned_models/{language}_{tokenizer_param.get_value()}',
      num_labels=n_labels_param.get_value(),
      ignore_mismatched_sizes=True
  )

  test = pd.read_csv(f'subtask1/dev/{language}.csv')
  texts = test['text'].tolist()
  ids = test['id'].tolist()
  labels = np.zeros(len(texts)).tolist()
  # Create datasets
  test_dataset = PolarizationDataset(texts, labels, tokenizer)

  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      eval_dataset=test_dataset,
      compute_metrics=subtask1_codabench_compute_metrics,     # the callback that computes metrics of interest
      data_collator=DataCollatorWithPadding(tokenizer) # Data collator for dynamic padding
  )

  predictions = trainer.predict(test_dataset)
  pred_labels = np.argmax(predictions.predictions, axis=1)

  results_df = pd.DataFrame({
      'id': ids,
      'polarization': pred_labels,
  })
  results_df.to_csv(f'results/subtask_1/pred_{language}.csv', index=False)


A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


In [ ]:
! cd results && zip -r ../subtask_1_{experiment_version}.zip subtask_1 && cd ..
from google.colab import files
files.download(f'subtask_1_{experiment_version}.zip')

  adding: subtask_1/ (stored 0%)
  adding: subtask_1/pred_amh.csv (deflated 48%)
  adding: subtask_1/pred_swa.csv (deflated 48%)
  adding: subtask_1/pred_eng.csv (deflated 48%)
  adding: subtask_1/pred_hau.csv (deflated 48%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Contrastive Multilingual Modelling

In [ ]:
# @title Update Parameters 4 Contrastive Alignment

anchor_model = 'olateju/PolarPairs-v1.0.3_eng' # @param
anchor_model_param = Parameter(anchor_model, 'AnchorModel', None)

num_epochs = 3 # @param
epochs_param = Parameter(num_epochs, 'epochs', 'Hyperparameter')

train_batch = 8 # @param
train_batch_param = Parameter(train_batch, 'train_batch_size', 'Hyperparameter')

eval_batch = 8 # @param
eval_batch_param = Parameter(eval_batch, 'eval_batch_size', 'Hyperparameter')

contrastive_subset = 2 # @param
contrastive_subset_param = Parameter(
    contrastive_subset,
    'subset_size',
    'Hyperparameter'
)

contrastive_temperature = 0.07 # @param
contrastive_temperature_param = Parameter(
    contrastive_temperature,
    'tau',
    'Hyperparameter'
)

lambda_align = 0.1 # @param
lambda_align_param = Parameter(
    lambda_align,
    'lambda',
    'Hyperparameter'
)

encoder_lr = 1e-4 # @param
encoder_lr_param = Parameter(
    encoder_lr,
    'encoder_lr',
    'Hyperparameter'
)

alignment_lr = 5e-3 # @param
alignment_lr_param = Parameter(
    alignment_lr,
    'alignment_lr',
    'Hyperparameter'
)

classification_lr = 5e-3 # @param
classification_lr_param = Parameter(
    classification_lr,
    'encoder_lr',
    'Hyperparameter'
)

alignment_resnet = False # @param {type: 'boolean'}
alignment_resnet_param = Parameter(
    alignment_resnet,
    'alignment_resnet',
    'Hyperparameter'
)

alignment_normalization = False # @param {type: 'boolean'}
alignment_normalization_param = Parameter(
    alignment_normalization,
    'alignment_normalization',
    'Hyperparameter'
)

experiment.add_params([
    anchor_model_param, epochs_param, train_batch_param, eval_batch_param,
    contrastive_subset_param, contrastive_temperature_param,
    lambda_align_param,
    encoder_lr_param, alignment_lr_param, classification_lr_param,
    alignment_resnet_param, alignment_normalization_param
])

In [ ]:
# @title Update Training Arguments

# Training setup
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs_param.get_value(),
    per_device_train_batch_size=train_batch_param.get_value(),  # mT5-small can handle 2-4
    per_device_eval_batch_size=eval_batch_param.get_value(),
    gradient_accumulation_steps=8,
    eval_strategy=eval_strategy_param.get_value(),
    save_strategy="no",
    logging_steps=50,
    fp16=True,
    dataloader_num_workers=0,
    load_best_model_at_end=False,
    eval_accumulation_steps=1,
    gradient_checkpointing=False,
)

In [ ]:
# @title PolarPairsAlignment Training

from torch.optim import AdamW
np.random.seed(42)
torch.manual_seed(42)

SRC_LANG = 'hau'
TGT_LANG = 'eng'

train, val = load_and_split_bilingual_data(
      subtask = 'subtask1',
      source_lang = SRC_LANG,
      target_lang = TGT_LANG
)
val['target_text'] = val['source_text']
val['target_polarization'] = val['polarization']

# Initialize dataset
train_dataset = PolarPairsDataset(
    source_texts = train['source_text'].tolist(),
    target_texts = train['target_text'].tolist(),  # or different language
    source_labels = train['polarization'].tolist(),
    target_labels = train['target_polarization'].tolist(),  # labels for targets
    source_tokenizer = tokenizer_param.get_value(),
    target_tokenizer = anchor_model_param.get_value(),
    max_length = max_length_param.get_value(),
    subset_size = contrastive_subset_param.get_value()
)

val_dataset = PolarPairsDataset(
    source_texts = val['source_text'].tolist(),
    target_texts = val['target_text'].tolist(),  # or different language
    source_labels = val['polarization'].tolist(),
    target_labels = val['target_polarization'].tolist(),  # labels for targets
    source_tokenizer = tokenizer_param.get_value(),
    target_tokenizer = anchor_model_param.get_value(),
    max_length = max_length_param.get_value(),
    subset_size = contrastive_subset_param.get_value()
)

# Initialize model
model = MultiLingualPolarPairsAlignment(
    encoder_model_name=tokenizer_param.get_value(),
    pretrained_encoder_name=anchor_model_param.get_value(),  # or 'bert-base-multilingual-cased'
    num_labels=n_labels_param.get_value(),
    alignment_normalization=alignment_normalization_param.get_value(),
    alignment_residual=alignment_resnet_param.get_value(),
)

optimizer = AdamW([
    {'params': model.encoder.parameters(), 'lr': encoder_lr_param.get_value()},
    # {'params': model.translation_encoder.parameters(), 'lr': 1e-4},
    {'params': model.alignment_head.parameters(), 'lr': alignment_lr_param.get_value()},
    {'params': model.classification_head.parameters(), 'lr': classification_lr_param.get_value()}
])

# Freeze encoders (only train alignment + classification heads)
# for param in model.encoder.parameters():
#     param.requires_grad = False
# for param in model.translation_encoder.parameters():
#     param.requires_grad = False

collator = PolarPairsCollator()

# Train
trainer = PolarPairsTrainer(
    lambda_align = lambda_align_param.get_value(),
    temperature=contrastive_temperature_param.get_value(),
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics,
    data_collator = collator,
    optimizers = (optimizer, None)
)

trainer.train()

target_results = trainer.evaluate()
ft_results = {
    'src_lang': SRC_LANG,
    'tgt_lang': TGT_LANG,
    'src_performance': target_results
}
_results_param = Parameter(ft_results, f"{SRC_LANG}_multilingual+FT_results", "Performance")
experiment.add_params([_results_param])

# ===== SAVE THE FINE-TUNED MODEL =====
# save_path = f'finetuned_models/{TGT_LANG}-{SRC_LANG}_{tokenizer_param.get_value()}'
# model.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)
# print(f"Saved {TGT_LANG}-{SRC_LANG} model to {save_path}")
# ====================================

# ======================================
# Evaluate for Target Language English
# ======================================

val = pd.read_csv(f'subtask1/train/eng.csv')

# Initialize dataset
val_dataset = PolarPairsDataset(
    source_texts = val['text'].tolist(),
    target_texts = val['text'].tolist(),  # or different language
    source_labels = val['polarization'].tolist(),
    target_labels = val['polarization'].tolist(),  # labels for targets
    source_tokenizer = tokenizer_param.get_value(),
    target_tokenizer = anchor_model_param.get_value(),
    max_length = max_length_param.get_value(),
    subset_size = contrastive_subset_param.get_value()
)

trainer = PolarPairsTrainer(
    lambda_align = lambda_align_param.get_value(),
    temperature=contrastive_temperature_param.get_value(),
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics,
    data_collator = collator,
    optimizers = (optimizer, None)
)

target_results = trainer.evaluate()
ft_results = {
    'src_lang': TGT_LANG,
    'tgt_lang': TGT_LANG,
    'src_performance': target_results
}
_results_param = Parameter(ft_results, f"{SRC_LANG}+{TGT_LANG}_multilingual+FT_results", "Performance")
experiment.add_params([_results_param])

Dataset: subtask1 | hau → eng
Total samples: 3651
Train set size: 2920 (80.0%)
Validation set size: 731 (20.0%)

--- Polarization Distribution ---

Train Set (Source):
polarization
0    2606
1     314
Name: count, dtype: int64
  Ratio: {0: 0.8924657534246575, 1: 0.10753424657534247}

Validation Set (Source):
polarization
0    653
1     78
Name: count, dtype: int64
  Ratio: {0: 0.893296853625171, 1: 0.106703146374829}

Validation Set (Target):
target_polarization
0    496
1    235
Name: count, dtype: int64
  Ratio: {0: 0.6785225718194254, 1: 0.32147742818057456}
Target distribution - Positive: 939, Negative: 1981
Target distribution - Positive: 78, Negative: 653


Epoch,Training Loss,Validation Loss,F1 Macro,F1 Weighted
1,No log,0.373801,0.826262,0.930581
2,0.580300,0.361733,0.787891,0.909999
3,0.342800,0.366413,0.812564,0.927326


Target distribution - Positive: 1175, Negative: 2047


In [ ]:

class CustomDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        batch = super().__call__(features)
        # Convert labels to long (int64)
        if 'labels' in batch:
            batch['labels'] = batch['labels'].long()
        return batch

In [ ]:
# @title Test Base English Model on both languages

! mkdir -p multilingual_results

from torch.utils.data import DataLoader
from tqdm import tqdm

for language in ['swa']:
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_param.get_value())
  model = AutoModelForSequenceClassification.from_pretrained(
      f'finetuned_models/eng_{tokenizer_param.get_value()}',
      num_labels=n_labels_param.get_value()
  )

  test = pd.read_csv(f'subtask1/train/{language}.csv')
  texts = test['text'].tolist()
  ids = test['id'].tolist()
  labels = np.zeros(len(texts)).tolist()
  # Create datasets
  test_dataset = PolarizationDataset(texts, labels, tokenizer)

  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      eval_dataset=test_dataset,
      compute_metrics=subtask1_codabench_compute_metrics,     # the callback that computes metrics of interest
      data_collator=DataCollatorWithPadding(tokenizer) # Data collator for dynamic padding
  )

  eval_results = trainer.evaluate()
  print(f"Macro F1 score for eng model on {language}: {eval_results['eval_f1_macro']}")

  eval_results_param = Parameter(eval_results, f"eng({language})_eval_results", "Performance")
  experiment.add_params([eval_results_param])

Macro F1 score for eng model on swa: 0.44201452629898635


#### Knowledge Distillation

In [ ]:
# @title Experiment Definition & Parameter Setting

import os

student_model_name = "distilbert-base-multilingual-cased" # @param {type: "string"}
student_model_param = Parameter(student_model_name, 'student_model', None)

teacher_model_name = "sentence-transformers/LaBSE" # @param {type: "string"}
teacher_model_param = Parameter(teacher_model_name, 'teacher_model', None)

max_length = 128 # @param
max_length_param = Parameter(max_length, 'max_length', 'Hyperparameter')

num_epochs = 3 # @param
epochs_param = Parameter(num_epochs, 'epochs', 'Hyperparameter')

lr = 1e-4 # @param
lr_param = Parameter(lr, 'learning_rate', 'Hyperparameter')

train_batch = 8 # @param
train_batch_param = Parameter(train_batch, 'train_batch_size', 'Hyperparameter')

eval_batch = 8 # @param
eval_batch_param = Parameter(eval_batch, 'eval_batch_size', 'Hyperparameter')

eval_strategy = "epoch" # @param {type: "string"}
eval_strategy_param = Parameter(eval_strategy, 'eval_strategy', 'Hyperparameter')

experiment_version = "v1.3.6.3" # @param {type: "string"}
experiment_dir = f"experiments/{experiment_version}.yaml"
if os.path.exists(experiment_dir):
  raise ValueError(f"Experiment {experiment_version} already exists")
experiment_description = "Distilling LaBSE embeddings for selected African Languages of FLORES-200 dataset to distilled-BERT" # @param {type: "string"}
experiment_baseline = "v1.2.6.3" # @param {type: "string"}
experiment = Experiment(
    experiment_version,
    experiment_dir,
    experiment_description,
    experiment_baseline
)
experiment.add_params([
    student_model_param, teacher_model_param,
    max_length_param, epochs_param, lr_param,
    train_batch_param, eval_batch_param,
    eval_strategy_param
])

save_models = False # @param {type: 'boolean'}

In [ ]:
teacher_model = AutoModel.from_pretrained(teacher_model_param.get_value())
student_model = AutoModel.from_pretrained(student_model_param.get_value())

teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_param.get_value())
student_tokenizer = AutoTokenizer.from_pretrained(student_model_param.get_value())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model.to(device)
student_model.to(device)

# Load FLORES data
print("\nLoading FLORES-200 data...")
sentences = load_flores_multilingual()

if len(sentences) == 0:
  raise ValueError("❌ No sentences were loaded! Check your internet connection.")
print(f"✓ Successfully loaded {len(sentences)} sentences")

# Create dataset
# train_dataset = SentenceDataset(
#     sentences,
#     tokenizer,
#     max_length=max_length_param.get_value()
# )

# Create dataset with dual tokenizers
train_dataset = DualTokenizerDataset(
    sentences,
    teacher_tokenizer,
    student_tokenizer,
    max_length=max_length_param.get_value()
)

# Training arguments
training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = epochs_param.get_value(),
    per_device_train_batch_size = train_batch_param.get_value(),
    learning_rate = lr_param.get_value(),
    warmup_steps = 500,
    logging_steps = 100,
    save_steps = 1000,
    save_total_limit = 2,
    fp16 = torch.cuda.is_available(),
    dataloader_num_workers = 2,
    remove_unused_columns = False,
    disable_tqdm = False
)

trainer = EmbeddingDistillationTrainer(
    model = student_model,
    args = training_args,
    train_dataset = train_dataset,
    teacher_model = teacher_model,
)

trainer.train()

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]


Loading FLORES-200 data...
Loading FLORES-200 dataset (this may take a moment)...
Error with 'all' method: Dataset scripts are no longer supported, but found flores.py

Trying alternative loading method...


Resolving data files:   0%|          | 0/224 [00:00<?, ?it/s]

...

Resolving data files:   0%|          | 0/218 [00:00<?, ?it/s]

Loaded 997 sentences for eng_Latn (dev)
Loaded 1012 sentences for eng_Latn (devtest)

Total sentences: 18081
✓ Successfully loaded 18081 sentences


A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


Step,Training Loss
100,0.001800
200,0.001000
300,0.000900
400,0.000900
500,0.000900
600,0.000900
700,0.000900
800,0.000900
900,0.000800
1000,0.000800


TrainOutput(global_step=6783, training_loss=0.0006749857501014924, metrics={'train_runtime': 1013.4167, 'train_samples_per_second': 53.525, 'train_steps_per_second': 6.693, 'total_flos': 1771689955295232.0, 'train_loss': 0.0006749857501014924, 'epoch': 3.0})

In [ ]:
# if save_models:
student_model.push_to_hub(f"olateju/PolarPairs-{experiment_version}")
student_tokenizer.push_to_hub(f"olateju/PolarPairs-{experiment_version}")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...oipxyai/model.safetensors:   0%|          |  559kB /  539MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/olateju/PolarPairs-v1.3.6.3/commit/2cc8269f75e737873b41dab8e84936cbe0a7f70b', commit_message='Upload tokenizer', commit_description='', oid='2cc8269f75e737873b41dab8e84936cbe0a7f70b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/olateju/PolarPairs-v1.3.6.3', endpoint='https://huggingface.co', repo_type='model', repo_id='olateju/PolarPairs-v1.3.6.3'), pr_revision=None, pr_num=None)

#### Save Experiment

In [ ]:
# @title Update experiment description

experiment_description = "Matching Encoder Pairs: distilled-LaBSE from LaBSE on African languages as slave-model and english optimized LABSE as anchor-model" # @param {type: "string"}
experiment.description = experiment_description

In [ ]:
experiment.parameters['Performance']

{'swa_multilingual+FT_results': {'src_lang': 'swa',
  'tgt_lang': 'eng',
  'src_performance': {'eval_loss': 0.5237382054328918,
   'eval_f1_macro': 0.7598273444812773,
   'eval_f1_weighted': 0.7598284488920658,
   'eval_runtime': 5.3374,
   'eval_samples_per_second': 262.115,
   'eval_steps_per_second': 32.788,
   'epoch': 3.0}},
 'swa+eng_multilingual+FT_results': {'src_lang': 'eng',
  'tgt_lang': 'eng',
  'src_performance': {'eval_loss': 0.9044691920280457,
   'eval_model_preparation_time': 0.0024,
   'eval_f1_macro': 0.5660349055625986,
   'eval_f1_weighted': 0.5939935155304985,
   'eval_runtime': 12.6382,
   'eval_samples_per_second': 254.941,
   'eval_steps_per_second': 31.887}},
 'amh_multilingual+FT_results': {'src_lang': 'amh',
  'tgt_lang': 'eng',
  'src_performance': {'eval_loss': 0.6604948043823242,
   'eval_f1_macro': 0.5593204341670599,
   'eval_f1_weighted': 0.6194441240918003,
   'eval_runtime': 2.5108,
   'eval_samples_per_second': 265.657,
   'eval_steps_per_second': 3

In [ ]:
# @title Save Model or not

# model.push_to_hub(f"olateju/PolarPairs-{experiment_version}")
# tokenizer.push_to_hub(f"olateju/PolarPairs-{experiment_version}")

In [ ]:
# experiment.dir = f"experiments/{experiment_version}.yaml"
experiment.save()

Model saved to experiments/v1.3.3.3.yaml


In [ ]:
! zip NLP_LLMS_experiments_{experiment_version}.zip experiments/*
from google.colab import files
files.download(f'NLP_LLMS_experiments_{experiment_version}.zip')

  adding: experiments/v1.3.3.3.yaml (deflated 70%)
  adding: experiments/v1.3.3.6.yaml (deflated 70%)
  adding: experiments/v1.3.6.3.yaml (deflated 70%)
  adding: experiments/v1.3.6.6.yaml (deflated 70%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Subtask 2: Polarization Type Classification
Multi-label classification to identify the target of polarization as one of the following categories: Gender/Sexual, Political, Religious, Racial/Ethnic, or Other.
For this task we will load the data for subtask 2.

In [ ]:
# @title Experiment Definition & Parameter Setting

import os

languages = ['eng', 'swa'] # @param
languages_param = Parameter(languages, 'language', None)

model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest" # @param {type: "string"}
model_param = Parameter(model_name, 'model_name', None)

tokenizer_choice = "cardiffnlp/twitter-roberta-base-sentiment-latest" # @param {type:"string"}
tokenizer_param = Parameter(tokenizer_choice, 'tokenizer', 'Preprocessing')

n_labels = 5 # @param
n_labels_param = Parameter(n_labels, 'n_labels', None)

max_length = 128 # @param
max_length_param = Parameter(max_length, 'max_length', 'Hyperparameter')

num_epochs = 3 # @param
epochs_param = Parameter(num_epochs, 'epochs', 'Hyperparameter')

lr = 2e-5 # @param
lr_param = Parameter(lr, 'learning_rate', 'Hyperparameter')

train_batch = 8 # @param
train_batch_param = Parameter(train_batch, 'train_batch_size', 'Hyperparameter')

eval_batch = 8 # @param
eval_batch_param = Parameter(eval_batch, 'eval_batch_size', 'Hyperparameter')

eval_strategy = "epoch" # @param {type: "string"}
eval_strategy_param = Parameter(eval_strategy, 'eval_strategy', 'Hyperparameter')

experiment_version = "v2.0.3" # @param {type: "string"}
experiment_dir = f"experiments/{experiment_version}.yaml"
if os.path.exists(experiment_dir):
  raise ValueError(f"Experiment {experiment_version} already exists")
experiment_description = "MultiLabel Sentiment using cardiffNLP robrerta twitter sentiment" # @param {type: "string"}
experiment_baseline = "None" # @param {type: "string"}
experiment = Experiment(
    experiment_version,
    experiment_dir,
    experiment_description,
    experiment_baseline
)
experiment.add_params([
    languages_param, model_param, max_length_param,
    tokenizer_param, epochs_param, lr_param,
    train_batch_param, eval_batch_param,
    eval_strategy_param
])

In [ ]:
# Fix the dataset class by inheriting from torch.utils.data.Dataset
class PolarizationDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length # Store max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding=False, max_length=self.max_length, return_tensors='pt')

        # Ensure consistent tensor conversion for all items
        item = {key: encoding[key].squeeze() for key in encoding.keys()}
        # CHANGE THIS LINE: Use torch.float instead of torch.long for multi-label classification
        item['labels'] = torch.tensor(label, dtype=torch.float)
        return item


In [ ]:
# @title Define training arguments
training_args = TrainingArguments(
    output_dir=f"./",
    num_train_epochs=epochs_param.get_value(),
    learning_rate=lr_param.get_value(),
    per_device_train_batch_size=train_batch_param.get_value(),
    per_device_eval_batch_size=eval_batch_param.get_value(),
    eval_strategy=eval_strategy_param.get_value(),
    save_strategy="no",
    logging_steps=100,
    disable_tqdm=False
)

In [ ]:
for language in languages_param.get_value():

  clear_output()

  data = pd.read_csv(f'subtask2/train/{language}.csv')
  train_stop_idx = int(0.8*len(data))
  train = data[0:train_stop_idx]
  val = data[train_stop_idx:]

  # Load the tokenizer
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_param.get_value())

  # Create train and Test dataset for multilabel
  train_dataset = PolarizationDataset(train['text'].tolist(), train[['gender/sexual','political','religious','racial/ethnic','other']].values.tolist(), tokenizer)
  val_dataset = PolarizationDataset(val['text'].tolist(), val[['gender/sexual','political','religious','racial/ethnic','other']].values.tolist(), tokenizer)

  model = AutoModelForSequenceClassification.from_pretrained(
      model_param.get_value(),
      num_labels=n_labels_param.get_value(),
      problem_type="multi_label_classification",
      ignore_mismatched_sizes=True
  )

  # Initialize the Trainer
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      compute_metrics=subtask2_codabench_compute_metrics_multilabel,  # Use the new metrics function
      data_collator=DataCollatorWithPadding(tokenizer)
  )

  # Train the model
  trainer.train()

  # Evaluate the model on the validation set
  eval_results = trainer.evaluate()
  print(f"Macro F1 score on validation set for Subtask 2: {eval_results['eval_f1_macro']}")

  # ===== SAVE THE FINE-TUNED MODEL =====
  save_path = f'finetuned_models/subtask2_{language}_{tokenizer_param.get_value()}'
  model.save_pretrained(save_path)
  tokenizer.save_pretrained(save_path)
  print(f"Saved {language} model to {save_path}")
  # ====================================

  eval_results_param = Parameter(eval_results, f"{language}_eval_results", "Performance")
  experiment.add_params([eval_results_param])

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

Epoch,Training Loss,Validation Loss,F1 Macro
1,0.187400,0.313073,0.095962
2,0.147700,0.332889,0.106736
3,0.143900,0.331292,0.119158


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Macro F1 score on validation set for Subtask 2: 0.11915785250158821
Saved swa model to finetuned_models/subtask2_swa_cardiffnlp/twitter-roberta-base-sentiment-latest


In [ ]:
experiment.save()

Model saved to experiments/v2.0.3.yaml


In [ ]:
# @title Codabench Submission

! mkdir -p results/subtask_2/

from torch.utils.data import DataLoader
from tqdm import tqdm

for language in languages_param.get_value():
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_param.get_value())
  model = AutoModelForSequenceClassification.from_pretrained(
      f'finetuned_models/subtask2_{language}_{tokenizer_param.get_value()}',
      num_labels=n_labels_param.get_value()
  )

  test = pd.read_csv(f'subtask2/dev/{language}.csv')
  texts = test['text'].tolist()
  ids = test['id'].tolist()
  labels = np.zeros((len(texts), n_labels_param.get_value())).tolist()
  # Create datasets
  test_dataset = PolarizationDataset(texts, labels, tokenizer)

  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      eval_dataset=test_dataset,
      compute_metrics=subtask2_codabench_compute_metrics_multilabel,     # the callback that computes metrics of interest
      data_collator=DataCollatorWithPadding(tokenizer) # Data collator for dynamic padding
  )

  predictions = trainer.predict(test_dataset)
  probs = torch.sigmoid(torch.from_numpy(predictions.predictions))
  preds = (probs > 0.5).int().numpy()

  # 'gender/sexual','political','religious','racial/ethnic','other']

  results_df = pd.DataFrame({
      'id': ids,
      'gender/sexual': preds[:, 0],
      'political': preds[:, 1],
      'religious': preds[:, 2],
      'racial/ethnic': preds[:, 3],
      'other': preds[:, 4]
  })
  results_df.to_csv(f'results/subtask_2/pred_{language}.csv', index=False)


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


In [ ]:
! cd results && zip -r ../subtask_2.zip subtask_2 && cd ..
from google.colab import files
files.download('subtask_2.zip')

  adding: subtask_2/ (stored 0%)
  adding: subtask_2/pred_eng.csv (deflated 55%)
  adding: subtask_2/pred_swa.csv (deflated 56%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
! zip NLP_LLMS_subtask2_experiments.zip experiments/*
from google.colab import files
files.download('NLP_LLMS_subtask2_experiments.zip')

  adding: experiments/v2.0.2.yaml (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Subtask 3: Manifestation Identification
Multi-label classification to classify how polarization is expressed, with multiple possible labels including Vilification, Extreme Language, Stereotype, Invalidation, Lack of Empathy, and Dehumanization.



In [ ]:
# @title Experiment Definition & Parameter Setting

import os

languages = ['eng', 'swa'] # @param
languages_param = Parameter(languages, 'language', None)

model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest" # @param {type: "string"}
model_param = Parameter(model_name, 'model_name', None)

tokenizer_choice = "cardiffnlp/twitter-roberta-base-sentiment-latest" # @param {type:"string"}
tokenizer_param = Parameter(tokenizer_choice, 'tokenizer', 'Preprocessing')

n_labels = 6 # @param
n_labels_param = Parameter(n_labels, 'n_labels', None)

max_length = 128 # @param
max_length_param = Parameter(max_length, 'max_length', 'Hyperparameter')

num_epochs = 3 # @param
epochs_param = Parameter(num_epochs, 'epochs', 'Hyperparameter')

lr = 2e-5 # @param
lr_param = Parameter(lr, 'learning_rate', 'Hyperparameter')

train_batch = 8 # @param
train_batch_param = Parameter(train_batch, 'train_batch_size', 'Hyperparameter')

eval_batch = 8 # @param
eval_batch_param = Parameter(eval_batch, 'eval_batch_size', 'Hyperparameter')

eval_strategy = "epoch" # @param {type: "string"}
eval_strategy_param = Parameter(eval_strategy, 'eval_strategy', 'Hyperparameter')

experiment_version = "v3.0.2" # @param {type: "string"}
experiment_dir = f"experiments/{experiment_version}.yaml"
if os.path.exists(experiment_dir):
  raise ValueError(f"Experiment {experiment_version} already exists")
experiment_description = "Manifestation identification using cardiggNLP twitter roberta sentiment analysis" # @param {type: "string"}
experiment_baseline = "None" # @param {type: "string"}
experiment = Experiment(
    experiment_version,
    experiment_dir,
    experiment_description,
    experiment_baseline
)
experiment.add_params([
    languages_param, model_param, max_length_param,
    tokenizer_param, epochs_param, lr_param,
    train_batch_param, eval_batch_param,
    eval_strategy_param
])

In [ ]:
# train = pd.read_csv('subtask3/train/eng.csv')
# val = pd.read_csv('subtask3/train/eng.csv')

# train.head()

In [ ]:
# Fix the dataset class by inheriting from torch.utils.data.Dataset
class PolarizationDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length # Store max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding=False, max_length=self.max_length, return_tensors='pt')

        # Ensure consistent tensor conversion for all items
        item = {key: encoding[key].squeeze() for key in encoding.keys()}
        # CHANGE THIS LINE: Use torch.float instead of torch.long for multi-label classification
        item['labels'] = torch.tensor(label, dtype=torch.float)
        return item

In [ ]:
# @title Define training arguments

training_args = TrainingArguments(
    output_dir=f"./",
    num_train_epochs=epochs_param.get_value(),
    learning_rate=lr_param.get_value(),
    per_device_train_batch_size=train_batch_param.get_value(),
    per_device_eval_batch_size=eval_batch_param.get_value(),
    eval_strategy=eval_strategy_param.get_value(),
    save_strategy="no",
    logging_steps=100,
    disable_tqdm=False
)

In [ ]:
# Define metrics function for multi-label classification
def compute_metrics_multilabel(p):
    # Sigmoid the predictions to get probabilities
    probs = torch.sigmoid(torch.from_numpy(p.predictions))
    # Convert probabilities to predicted labels (0 or 1)
    preds = (probs > 0.5).int().numpy()
    # Compute macro F1 score
    return {'f1_macro': f1_score(p.label_ids, preds, average='macro')}

In [ ]:
# Initialize the Trainer

for language in languages_param.get_value():

  clear_output()

  data = pd.read_csv(f'subtask3/train/{language}.csv')
  train_stop_idx = int(0.8*len(data))
  train = data[0:train_stop_idx]
  val = data[train_stop_idx:]

  data = pd.read_csv(f'subtask3/train/{language}.csv')
  train_stop_idx = int(0.8*len(data))
  train = data[0:train_stop_idx]
  val = data[train_stop_idx:]

  # Load the tokenizer
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_param.get_value())

  # Create train and Test dataset for multilabel
  train_dataset = PolarizationDataset(train['text'].tolist(), train[['vilification','extreme_language','stereotype','invalidation','lack_of_empathy','dehumanization']].values.tolist(), tokenizer)
  val_dataset = PolarizationDataset(val['text'].tolist(), val[['vilification','extreme_language','stereotype','invalidation','lack_of_empathy','dehumanization']].values.tolist(), tokenizer)

  model = AutoModelForSequenceClassification.from_pretrained(
      model_param.get_value(),
      num_labels=n_labels_param.get_value(),
      problem_type="multi_label_classification",
      ignore_mismatched_sizes=True
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      compute_metrics=compute_metrics_multilabel,  # Use the new metrics function
      data_collator=DataCollatorWithPadding(tokenizer)
  )

  # Train the model
  trainer.train()

  # Evaluate the model on the validation set
  eval_results = trainer.evaluate()
  print(f"Macro F1 score on validation set for Subtask 3: {eval_results['eval_f1_macro']}")

  # ===== SAVE THE FINE-TUNED MODEL =====
  save_path = f'finetuned_models/subtask3_{language}_{tokenizer_param.get_value()}'
  model.save_pretrained(save_path)
  tokenizer.save_pretrained(save_path)
  print(f"Saved {language} model to {save_path}")
  # ====================================

  eval_results_param = Parameter(eval_results, f"{language}_eval_results", "Performance")
  experiment.add_params([eval_results_param])

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

Epoch,Training Loss,Validation Loss,F1 Macro
1,0.503400,0.662416,0.168447
2,0.475800,0.561850,0.163097
3,0.443600,0.665743,0.167027


Macro F1 score on validation set for Subtask 3: 0.16702704434224694
Saved swa model to finetuned_models/subtask3_swa_cardiffnlp/twitter-roberta-base-sentiment-latest


In [ ]:
# @title Codabench Submission

! mkdir -p results/subtask_3/

from torch.utils.data import DataLoader
from tqdm import tqdm

for language in languages_param.get_value():
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_param.get_value())
  model = AutoModelForSequenceClassification.from_pretrained(
      f'finetuned_models/subtask3_{language}_{tokenizer_param.get_value()}',
      num_labels=n_labels_param.get_value()
  )

  test = pd.read_csv(f'subtask3/dev/{language}.csv')
  texts = test['text'].tolist()
  ids = test['id'].tolist()
  labels = np.zeros((len(texts), n_labels_param.get_value())).tolist()
  # Create datasets
  test_dataset = PolarizationDataset(texts, labels, tokenizer)

  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      eval_dataset=test_dataset,
      compute_metrics=subtask2_codabench_compute_metrics_multilabel,     # the callback that computes metrics of interest
      data_collator=DataCollatorWithPadding(tokenizer) # Data collator for dynamic padding
  )

  predictions = trainer.predict(test_dataset)
  probs = torch.sigmoid(torch.from_numpy(predictions.predictions))
  preds = (probs > 0.5).int().numpy()

  # 'vilification','extreme_language','stereotype','invalidation','lack_of_empathy','dehumanization'

  results_df = pd.DataFrame({
      'id': ids,
      'vilification': preds[:, 0],
      'extreme_language': preds[:, 1],
      'stereotype': preds[:, 2],
      'invalidation': preds[:, 3],
      'lack_of_empathy': preds[:, 4],
      'dehumanization': preds[:, 5]
  })
  results_df.to_csv(f'results/subtask_3/pred_{language}.csv', index=False)


A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


In [ ]:
experiment.save()

Model saved to experiments/v3.0.2.yaml


In [ ]:
! cd results && zip -r ../subtask_3.zip subtask_3 && cd ..
from google.colab import files
files.download('subtask_3.zip')

updating: subtask_3/ (stored 0%)
updating: subtask_3/pred_eng.csv (deflated 57%)
updating: subtask_3/pred_swa.csv (deflated 57%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
! zip NLP_LLMS_subtask3_experiments.zip experiments/*
from google.colab import files
files.download('NLP_LLMS_subtask3_experiments.zip')

  adding: experiments/v3.0.0.yaml (deflated 53%)
  adding: experiments/v3.0.1.yaml (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>